In [1]:
conda install -c mosek mosek

Channels:
 - mosek
 - defaults
Platform: osx-64
Solving environment: done

## Package Plan ##

  environment location: /Users/alim/opt/anaconda3

  added / updated specs:
    - mosek


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    mosek-10.1.29              |           py39_0        12.5 MB  mosek
    openssl-3.0.13             |       hca72f7f_1         4.6 MB
    ------------------------------------------------------------
                                           Total:        17.1 MB

The following NEW packages will be INSTALLED:

  mosek              mosek/osx-64::mosek-10.1.29-py39_0 

The following packages will be UPDATED:

  openssl                                 3.0.13-hca72f7f_0 --> 3.0.13-hca72f7f_1 



mosek-10.1.29        | 12.5 MB   |                                       |   0% 
openssl-3.0.13       | 4.6 MB    |                                       |   0% 
openssl-3

## you need to get MOSEK academic license as well

### Hello World!

In [1]:
from mosek import *

with Env() as env:                            # Create Environment
    with env.Task(0, 1) as task:                # Create Task
        task.appendvars(1)                          # 1 variable x
        task.putcj(0, 1.0)                          # c_0 = 1.0
        task.putvarbound(0, boundkey.ra, 2.0, 3.0)  # 2.0 <= x <= 3.0
        task.putobjsense(objsense.minimize)         # minimize

        task.optimize()                           # Optimize

        x = task.getxx(soltype.itr)               # Get solution
        print("Solution x = {}".format(x[0]))     # Print solution

Solution x = 2.0


### Least Squares

In [2]:
from mosek.fusion import *
def ls(Q, a):
    d, N = len(Q), len(Q[0])
    M = Model("LS")
    
    # The regression coefficients
    b = M.variable("b", N)
    
    # The bound on the norm of the residual
    e = M.variable("e")
    r = Expr.sub(a, Expr.mul(Q,b))
    # t \geq |r|_2
    M.constraint(Expr.vstack(e, r), Domain.inQCone())

    M.objective(ObjectiveSense.Minimize, e)
    M.solve()
    return M.getVariable("b").level(), M.getVariable("e").level()

In [4]:
import numpy as np
d = 100
N = 10000
m = 10
Q = np.random.rand(d, N)
#b_s=np.random.rand(N)
#b_s=b_s/sum(b_s)*m
a = np.random.rand(d)
b, e = ls(Q, a)
b_ols = np.linalg.lstsq(Q, a, rcond=None)[0]
print(np.linalg.norm(Q@b-a), np.linalg.norm(Q@b_ols-a), np.linalg.norm(b-b_ols))

2.263956315453949e-12 1.5568767351709187e-14 2.4718870318024777e-14


### Box-bound Least Squares

In [3]:
def bblse(Q, a):
    d, N = len(Q), len(Q[0])
    M = Model("BBLS")
    
    b = M.variable("b", N, Domain.inRange(0.0, 1.0))
    
    # The bound on the norm of the residual
    e = M.variable("e")
    r = Expr.sub(a, Expr.mul(Q,b))
    # t \geq |r|_2
    M.constraint(Expr.vstack(e, r), Domain.inQCone())

    M.objective(ObjectiveSense.Minimize, e)
    M.solve()
    return M.getVariable("b").level(), M.getVariable("e").level()

In [5]:
import numpy as np
d = 100
N = 10000
m = 10
Q = np.random.rand(d, N)
a = np.random.rand(d)
b, e = bblse(Q, a)
b_ols = np.linalg.lstsq(Q, a, rcond=None)[0]
print(np.linalg.norm(Q@b-a), np.linalg.norm(Q@b_ols-a), np.linalg.norm(b-b_ols))
print(max(b), min(b))

2.0677742642244885 1.2330535272520888e-14 0.2734823963603019
0.11526185282404991 -2.7238975337907723e-09


In [32]:
def sls(Q, a, m):
    d, N = len(Q), len(Q[0])
    M = Model("SLS")
    
    b = M.variable("b", N)
    
    # The bound on the norm of the residual
    e = M.variable("e")
    r = Expr.sub(a, Expr.mul(Q,b))
    # t \geq |r|_2
    M.constraint(Expr.vstack(e, r), Domain.inQCone())
    M.constraint(Expr.sum(b), Domain.equalsTo(m))

    M.objective(ObjectiveSense.Minimize, e)
    M.solve()
    return M.getVariable("b").level(), M.getVariable("e").level()

In [33]:
import numpy as np
d = 100
N = 10000
m = 10
Q = np.random.rand(d, N)
a = np.random.rand(d)
b, e = sls(Q, a, m)
b_ols = np.linalg.lstsq(Q, a, rcond=None)[0]
print(np.linalg.norm(Q@b-a), np.linalg.norm(Q@b_ols-a), np.linalg.norm(b-b_ols))
print(sum(b))

4.200527064686183e-12 6.049226442732378e-14 1.5460925782329118
10.000000000002549


### box-bounded + sum = m least squares

In [16]:
from mosek.fusion import *
import mosek.fusion
def bbsls(Q, a, m):
    Q = np.array(Q)
    a = np.array(a)
    
    d, N = len(Q), len(Q[0])
    print(Q.shape)
    print(a.shape)
    print(m)
    print(N)
    M = mosek.fusion.Model("BBSLS")
    
    b = M.variable("b", N, mosek.fusion.Domain.inRange(0.0, 1.0))
    
    # The bound on the norm of the residual
    e = M.variable("e")
    r = mosek.fusion.Expr.sub(a, mosek.fusion.Expr.mul(Q,b))
    # t \geq |r|_2
    M.constraint(mosek.fusion.Expr.vstack(e, r), mosek.fusion.Domain.inQCone())
    M.constraint(mosek.fusion.Expr.sum(b), mosek.fusion.Domain.equalsTo(m))
    
    M.objective(mosek.fusion.ObjectiveSense.Minimize, e)
    M.solve()
    return M.getVariable("b").level(), M.getVariable("e").level()

In [17]:
import numpy as np
d = 100
N = 10000
m = 10
Q = np.random.rand(d, N)
a = np.random.rand(d)
print(Q)
b, e = bbsls(Q, a, m)
b_ols = np.linalg.lstsq(Q, a, rcond=None)[0]
print(np.linalg.norm(Q@b-a), np.linalg.norm(Q@b_ols-a), np.linalg.norm(b-b_ols))
print(sum(b), max(b), min(b))

(100, 10000)
(100,)
10
10000
36.0931371474061 1.043647358860854e-14 2.5259966181198905
9.999999999995811 0.999999999996665 -2.5402612880838085e-13
